<a href="https://colab.research.google.com/github/apresland/tensorflow-convnets/blob/binary-classification/binary_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Binary classification 

## Overview
A standard demonstration of binary image classifiction that employs data augmentation and dropout layers to improve accuracy. Data augmentation is a technique that increases the diversity of your training data by applying random (but realistic) transformations such as image rotation.

## Setup

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import os
import zipfile

### Download the dataset
This tutorial uses the dogs_vs_cats dataset downloaded from TensorFlow Datasets
for convenience.

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/data.zip

zip_ref = zipfile.ZipFile('/tmp/cats_and_dogs_filtered.zip', 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

train_dir = '/tmp/cats_and_dogs_filtered/train'
validation_dir = '/tmp/cats_and_dogs_filtered/validation'

## Data pre-processing and data augmentation
We will augment training examples with random transformations. This replicates real-world examples helping prevent overfitting and helping the model generalize better. This can be done simply in Keras via the keras.preprocessing.image.ImageDataGenerator class.

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow training images in batches from train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

# Flow validation images in batches from test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

## Baseline convnet
Image classification is performed with a convnet, so we will train one as an initial baseline.

We create a small convnet with few layers and few filters per layer and combine this with data augmentation and dropout. Dropout reduces overfitting by randomly changing the input pattern to a layer analoguous to data augmentation. These both help to prevent the model learning random correlations.

The baseline model is a stack of 3 convolution layers with a ReLU activation and followed by max-pooling layers (similar to the architectures that Yann LeCun advocated in the 1990s for image classification).


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=50,
      validation_data=validation_generator,
      validation_steps=50)